# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [19]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import pickle

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table('disaster',con=engine)

In [3]:
X = df.message
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # tokenize text
    tokens = word_tokenize(text) 
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens
    pass

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier())),
        ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = model_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [7]:
model.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip...y=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))]))],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
              

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_test_pred = model.predict(X_test)

In [9]:
print(classification_report(y_test,y_test_pred,target_names=y.columns))

                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      4976
               request       0.80      0.43      0.56      1132
                 offer       0.00      0.00      0.00        31
           aid_related       0.74      0.59      0.66      2698
          medical_help       0.52      0.10      0.17       479
      medical_products       0.76      0.09      0.16       316
     search_and_rescue       0.60      0.08      0.14       197
              security       0.25      0.01      0.02       125
              military       0.58      0.08      0.14       218
                 water       0.84      0.32      0.46       417
                  food       0.81      0.39      0.52       731
               shelter       0.81      0.36      0.50       574
              clothing       0.80      0.08      0.14       105
                 money       0.80      0.03      0.05       147
        missing_people       0.67      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
accuracy = (y_test == y_test_pred).mean().mean()
print("Accuracy:", accuracy)

Accuracy: 0.942347801269


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
def model_pipeline2():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])
    
    parameters = {
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [20,30]
        }
    cv = GridSearchCV(pipeline, param_grid=parameters)
    return cv

In [12]:
model2 = model_pipeline2()
model2.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'features__text_pipeline__tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [20, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
y_test_pred = model2.predict(X_test)

In [14]:
def display_results(cv, y_test, y_pred):
    accuracy = (y_pred == y_test).mean().mean()

    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)
    print(classification_report(y_test,y_pred,target_names=y.columns))

In [15]:
display_results(model2,y_test,y_test_pred)

Accuracy: 0.945513622692

Best Parameters: {'clf__estimator__n_estimators': 30, 'features__text_pipeline__tfidf__use_idf': False}
                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      4976
               request       0.81      0.47      0.59      1132
                 offer       0.00      0.00      0.00        31
           aid_related       0.76      0.68      0.72      2698
          medical_help       0.54      0.10      0.18       479
      medical_products       0.76      0.11      0.19       316
     search_and_rescue       0.50      0.03      0.05       197
              security       0.00      0.00      0.00       125
              military       0.67      0.06      0.10       218
                 water       0.87      0.37      0.52       417
                  food       0.82      0.53      0.65       731
               shelter       0.79      0.36      0.50       574
              clothing       0.82    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
def model_pipeline3():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),

        ('clf', MultiOutputClassifier(GradientBoostingClassifier())),
        ])
    return pipeline

In [17]:
model3 = model_pipeline3()
model3.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
           n_jobs=1))])

In [18]:
y_test_pred = model3.predict(X_test)
print(classification_report(y_test,y_test_pred,target_names=y.columns))
accuracy = (y_test == y_test_pred).mean().mean()
print("Accuracy:", accuracy)

                        precision    recall  f1-score   support

               related       0.79      0.98      0.88      4976
               request       0.82      0.49      0.61      1132
                 offer       0.00      0.00      0.00        31
           aid_related       0.77      0.56      0.65      2698
          medical_help       0.55      0.21      0.30       479
      medical_products       0.68      0.29      0.40       316
     search_and_rescue       0.37      0.20      0.26       197
              security       0.12      0.08      0.10       125
              military       0.67      0.32      0.43       218
                 water       0.71      0.60      0.65       417
                  food       0.76      0.78      0.77       731
               shelter       0.78      0.61      0.68       574
              clothing       0.57      0.60      0.59       105
                 money       0.42      0.24      0.31       147
        missing_people       0.28      

### 9. Export your model as a pickle file

In [20]:
# Dump the trained decision tree classifier with Pickle
randomforest_pkl_filename = 'randomforest_classifier_20190118.pkl'
# Open the file to save as pkl file
randomforest_pkl_model_pkl = open(randomforest_pkl_filename, 'wb')
pickle.dump(model2, randomforest_pkl_model_pkl)
# Close the pickle instances
randomforest_pkl_model_pkl.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.